In [1]:
import sys
sys.path.append('..')
import H2_match_coeffs as gmc
import H2_ivp_coeff as gic
import utils.input_output as io
import numpy as np
import torch
import enr.H2 as energy
use_cuda = 1
torchdeviceId = torch.device('cuda:0') if use_cuda else 'cpu'
torchdtype = torch.float32
import os

In [2]:
path_dfaust_template = sys.path[0]+"/base/DFAUST_template.ply"
[v_template,faces,Fun0] = io.loadData(path_dfaust_template) 
v_template = torch.from_numpy(v_template).to(dtype=torchdtype, device=torchdeviceId)
faces = torch.from_numpy(faces).to(dtype=torch.long, device=torchdeviceId)

In [3]:
basis_np_mot = np.load(sys.path[0]+"/base/basis_DFAUST_motions.npy")
b_split= basis_np_mot.shape[0]
basis_np_shape = np.load(sys.path[0]+"/base/basis_DFAUST_shape.npy")
basis_np = np.concatenate((basis_np_mot,basis_np_shape),axis=0)

basis_torch_shape = torch.from_numpy(basis_np_shape).to(dtype=torchdtype, device=torchdeviceId)
basis_torch_mot = torch.from_numpy(basis_np_mot).to(dtype=torchdtype, device=torchdeviceId)
basis_torch = torch.from_numpy(basis_np).to(dtype=torchdtype, device=torchdeviceId)

print(b_split, basis_np.shape)

130 (170, 6890, 3)


In [4]:
a0=1
a1=1000
b1=100
c1=1
d1=1
a2=1

In [5]:
def getEndpointCoeff(filename):
    param1 = { 'weight_coef_dist': 10**1,'sig_geom':.4,'kernel_grass': 'linear',
              'max_iter': 500,'time_steps': 2}
    param2 = { 'weight_coef_dist': 10**5,'sig_geom':.1,'kernel_grass': 'linear',
              'max_iter': 500,'time_steps': 3}
    param3 = { 'weight_coef_dist': 10**10,'sig_geom':.025,'kernel_grass': 'linear',
              'max_iter': 1000,'time_steps': 5}
    paramlist=[param1,param2,param3]    

    [V_raw,F_raw,FunS] = io.loadData(filename)
    target = [V_raw,F_raw]
    init_path = torch.stack([v_template]*5,dim=0)
    chemin_exp,X = gmc.H2MultiRes_coeff(a0, a1, b1, c1, d1, a2, paramlist, [v_template.cpu().numpy(), faces.cpu().numpy()], target, init_path, faces, basis_torch)
    chemin_exp = init_path + torch.einsum("ij, jkl-> ikl",X, basis_torch)     
    return  X,[V_raw, F_raw]

In [6]:
path_to_target_endpoint0 = ""
path_to_target_endpoint1 = ""


X0, mesh0 = getEndpointCoeff(path_to_target_endpoint0)
X1, mesh1 = getEndpointCoeff(path_to_target_endpoint1)

stepX = X1[-1]-X0[-1]
steps=5
init_path = torch.stack([v_template]*steps,dim=0)

X,F=gic.H2InitialValueProblem(X0[-1].cpu().numpy(),steps*stepX.cpu().numpy(),steps,a0,a1,b1,c1,d1,a2,faces.cpu().numpy(),v_template,basis_torch,total_steps=steps)
geod = init_path + torch.einsum("ij, jkl-> ikl",X, basis_torch)
io.plotGeodesic(geod.cpu().numpy(),faces.cpu().numpy(),stepsize=2,axis=[0,1,0],angle=2*np.pi)